In [110]:
import pandas as pd
import os
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
import joblib

In [111]:
# Variables
features = ['perimeter', 'solidity', 'circularity', 'eccentricity', 'major_axis_length', 'minor_axis_length', 'Label']
continuous_features = ['perimeter', 'solidity', 'circularity', 'eccentricity', 'major_axis_length', 'minor_axis_length']
target_feature = 'Label'

TEST_SIZE = 0.25
RANDOM_STATE = 42

In [112]:
# Load the data
data = pd.read_csv('data/exported_features.csv')
data = data[features]

df = pd.DataFrame(data)

In [113]:
# Split target feature from the rest
X = df.drop('Label', axis=1)
y = df['Label']

In [114]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), continuous_features)
    ],
    remainder='drop'
)

# Define preprocessing pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
    ])

# Fit the pipeline
normalized_data = pipeline.fit_transform(df)

# Put results in a DataFrame
normalized_df = pd.DataFrame(normalized_data, columns=continuous_features)

# Encode the target feature
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Add the encoded target feature back
normalized_df['Label'] = y

# Add the original target feature back
normalized_df['Label_multiclass'] = df['Label']

normalized_df

,perimeter,solidity,circularity,eccentricity,major_axis_length,minor_axis_length,Label,Label_
0,-0.848487,-0.352934,0.896727,-1.837866,-1.333466,0.522725,1,r
1,-0.878911,-0.477691,0.852023,-1.990000,-1.418188,0.525102,1,r
2,-0.788467,-0.589404,0.761257,-1.764309,-1.167861,0.675744,1,r
3,-0.571321,-0.258747,0.754082,-0.268220,-0.281101,0.151155,1,r
4,-0.632256,-0.296963,0.858130,-1.909126,-1.233599,0.703428,1,r
...,...,...,...,...,...,...,...,...
128,0.666150,0.983861,-0.550939,0.728091,0.540697,-0.838676,2,s
129,1.093614,0.805906,-0.742775,0.962849,1.882337,-0.551481,2,s
130,0.491875,1.078796,-0.385757,0.792468,0.571487,-0.962210,2,s
131,0.385037,0.858401,-0.692802,0.658811,0.073702,-1.030649,2,s


In [115]:
# Export the scaler
joblib.dump(pipeline, '../models/deployed/preprocessing_pipeline.pkl')

['../models/deployed/preprocessing_pipeline.pkl']

In [116]:
# Export features to hand_gesture_features.csv
#Split the dataset into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(normalized_df.drop(target_feature, axis=1), normalized_df[target_feature], test_size=TEST_SIZE, random_state=RANDOM_STATE)

# Merge training data back together
train = pd.concat([train_X, train_y], axis=1)

# Merge testing data back together
test = pd.concat([test_X, test_y], axis=1)

# Define file paths
train_fp = '../models/data/hand_gesture_train.csv'
test_fp = '../models/data/hand_gesture_test.csv'

if not os.path.exists(train_fp):
    train.to_csv(train_fp, index=False)
    print(f"Created and saved new file: {train_fp}")
else:
    train.to_csv(train_fp, mode='w', header=True, index=False)
    print(f"Overwritten data to existing file: {train_fp}")
if not os.path.exists(test_fp):
    test.to_csv(test_fp, index=False)
    print(f"Created and saved new file: {test_fp}")
else:
    test.to_csv(test_fp, mode='w', header=True, index=False)
    print(f"Overwritten data to existing file: {test_fp}")

Overwritten data to existing file: ../models/data/hand_gesture_train.csv
Overwritten data to existing file: ../models/data/hand_gesture_test.csv
